In [1]:
# Installation cell
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.1-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

After you run the first cell (the the cell directly above this text), go to Colab's menu bar and select **Edit** and select **Notebook settings** from the drop down. Select *Julia 1.6* in Runtime type. You can also select your prefered harwdware acceleration (defaults to GPU). 

<br/>You should see something like this:

> ![Colab Img](https://raw.githubusercontent.com/Dsantra92/Julia-on-Colab/master/misc/julia_menu.png)

<br/>Click on SAVE
<br/>**We are ready to get going**





In [1]:
VERSION

v"1.6.1"

**The next three cells are for GPU benchmarking. If you are using this notebook for the first time and have GPU enabled, you can give it a try.** 

### Optional GPU Experiments

In [2]:
using Pkg
Pkg.add(["BenchmarkTools", "CUDA"])
using BenchmarkTools, CUDA

if has_cuda_gpu()
  print("The GPU device is:", CUDA.device())
end

The GPU device is:CuDevice(0)

In [3]:
mcpu = rand(2^10, 2^10)
@benchmark mcpu*mcpu

BenchmarkTools.Trial: 88 samples with 1 evaluation.
 Range (min … max):  52.154 ms … 69.716 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     56.043 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   56.984 ms ±  3.680 ms  ┊ GC (mean ± σ):  0.64% ± 1.57%

       ▁ ▃█▁▁▁▃▃▁▃▁ ▁                                          
  ▄▁▁▆▄█▆██████████▇█▇▁▇▁▇▁▆▄▄▁▄▄▁▁▁▁▁▁▁▁▁▄▄▁▁▁▁▁▁▁▁▁▁▁▁▆▄▄▁▄ ▁
  52.2 ms         Histogram: frequency by time        68.7 ms <

 Memory estimate: 8.00 MiB, allocs estimate: 2.

In [4]:
println("The CuArrray operation should take around 0.5 ms(excluding CUDA downloading time which is a one time process), and should be much faster. If so, the GPU is working.")
mgpu = cu(mcpu)
@benchmark CUDA.@sync mgpu*mgpu

The CuArrray operation should take around 0.5 ms(excluding CUDA downloading time which is a one time process), and should be much faster. If so, the GPU is working.


BenchmarkTools.Trial: 7673 samples with 1 evaluation.
 Range (min … max):  420.566 μs …  17.319 ms  ┊ GC (min … max): 0.00% … 42.11%
 Time  (median):     636.181 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   639.899 μs ± 261.709 μs  ┊ GC (mean ± σ):  0.28% ±  0.68%

                                     ▃▅▇██▅▄▂                    
  ▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▅▅▆▆▅▆▅▅▇█████████▆▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂ ▄
  421 μs           Histogram: frequency by time          761 μs <

 Memory estimate: 672 bytes, allocs estimate: 33.

In [14]:
using Pkg
Pkg.add(["Distributions"])
using Distributions

In [6]:
sigma=2
beta = 0.93
grd = 100
rho = 0.90
sigma_epsilon =0.3
r = 0.009
varphi = 0.6

0.6

In [7]:
sigma_z = sigma_epsilon/sqrt(1-rho^2)

0.6882472016116854

In [8]:
z_h = 3*sigma_z
z_l = -3*sigma_z
yc = 5

5

In [9]:
z=zeros(yc)


5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [12]:
midl = zeros(yc-1)
for i in 1:yc-1
    midl[i] = (z[i+1]+z[i])/2
end

In [15]:
Pi_y = zeros(yc, yc)
for i in 1: yc
    Pi_y[i,1] = cdf.(Normal(),(midl[1]-rho*z[i])/sigma_epsilon)
    Pi_y[i,yc]=1-cdf.(Normal(),(midl[yc-1]-rho*z[i])/sigma_epsilon)
    for j in 2:yc-1
    Pi_y[i,j]=cdf.(Normal(),(midl[j]-rho*z[i])/sigma_epsilon)-cdf.(Normal(),(midl[j-1]-rho*z[i]/sigma_epsilon))
    end
end

In [16]:
y = exp.(z)

5-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0

In [17]:
m = [0.000, 0.061,1.594]

3-element Vector{Float64}:
 0.0
 0.061
 1.594

In [18]:
Pi_m =[0.7 0.2 0.1; 0.15 0.65 0.2; 0.105 0.6601 0.233]

3×3 Matrix{Float64}:
 0.7    0.2     0.1
 0.15   0.65    0.2
 0.105  0.6601  0.233

In [19]:
mc = 3
#borrowing limits
bl = -(m[mc]-y[1])/r+20
bh =  120

120

In [27]:
bbl = (bl+200)^0.5
bbh = (bh+200)^0.5
st_l  =  (bbh-200^0.5)/(grd -1)
st_b  =  (200^0.5-bbl)/(grd -1)

0.017499615936768628

In [28]:
#grid for assets
b_l = zeros(grd)
b_b = zeros(grd)
for i in 1:grd
    b_l[i] = 200^0.5+(i-1)*st_l
    b_b[i] = bbl+(i-1)*st_b
end


In [29]:
a_l = b_l.^2 .- 200
a_b = b_b.^2 .- 200

100-element Vector{Float64}:
 -46.0
 -45.56536471803108
 -45.130116962946346
 -44.69425673474572
 -44.2577840334292
 -43.82069885899682
 -43.38300121144863
 -42.94469109078452
 -42.50576849700451
 -42.06623343010867
 -41.62608589009699
 -41.18532587696939
 -40.74395339072592
   ⋮
  -5.407548099191587
  -4.919015183026232
  -4.429869793745041
  -3.940111931347957
  -3.449741595834979
  -2.9587587872061363
  -2.4671635054614853
  -1.9749557506009126
  -1.4821355226244748
  -0.9887028215322289
  -0.4946576473240327
   2.842170943040401e-14

In [30]:
#grid for insurance
i = [0,1]


2-element Vector{Int64}:
 0
 1

In [31]:
# Utility function
utilb(a0,a1,y,i0,i1,m,∇)=((a0*(1+r)*∇+y-a1-prem*i1-(1-ϕ*i0)*m)^(1-σ)-1)/(1-σ)-λ*a0*(1-∇)
utill(a0,a1,y,i0,i1,m,∇)=((a0*(1+r)*δ+y-a1-prem*i1-(1-ϕ*i0)*m)^(1-σ)-1)/(1-σ)
consb(a0,a1,y,i0,i1,m,∇)= a0*(1+r)*∇+y-a1-prem*i1-(1-ϕ*i0)*m
consl(a0,a1,y,i0,i1,m,∇)= a0*(1+r)*δ+y-a1-prem*i1-(1-ϕ*i0)*m

consl (generic function with 1 method)

In [32]:
#Probability matrix
A=kron(Pi_m, Pi_y)
for i in 1:mc*yc
    A[i,:] = A[i,:]./sum(A[i,:])
end
B = ones(grd,grd)
D = kron(B,A)
C = ones(2,2)
Prob_l = kron(C, D)
E = ones(3,3)
Prob_b = kron(E,Prob_l)

9000×9000 Matrix{Float64}:
 0.35       0.0  0.0  0.0  0.35       …  0.05      0.0  0.0  0.0  0.05
 0.35       0.0  0.0  0.0  0.35          0.05      0.0  0.0  0.0  0.05
 0.35       0.0  0.0  0.0  0.35          0.05      0.0  0.0  0.0  0.05
 0.35       0.0  0.0  0.0  0.35          0.05      0.0  0.0  0.0  0.05
 0.35       0.0  0.0  0.0  0.35          0.05      0.0  0.0  0.0  0.05
 0.075      0.0  0.0  0.0  0.075      …  0.1       0.0  0.0  0.0  0.1
 0.075      0.0  0.0  0.0  0.075         0.1       0.0  0.0  0.0  0.1
 0.075      0.0  0.0  0.0  0.075         0.1       0.0  0.0  0.0  0.1
 0.075      0.0  0.0  0.0  0.075         0.1       0.0  0.0  0.0  0.1
 0.075      0.0  0.0  0.0  0.075         0.1       0.0  0.0  0.0  0.1
 0.0525999  0.0  0.0  0.0  0.0525999  …  0.116722  0.0  0.0  0.0  0.116722
 0.0525999  0.0  0.0  0.0  0.0525999     0.116722  0.0  0.0  0.0  0.116722
 0.0525999  0.0  0.0  0.0  0.0525999     0.116722  0.0  0.0  0.0  0.116722
 ⋮                                    ⋱  ⋮ 

In [35]:
#Vector of # values of a,m,i,y
nd  = [0, 0.5, 1] #∇ grid
H_b = kron(nd,ones(2*yc*mc*grd))
I_b = kron(ones(3),kron(i,ones(yc*mc*grd)))
A_b = kron(ones(3), kron(2,kron(a_b, ones(yc*mc))))
M_b = kron(ones(3), kron(2, kron(ones(grd), kron(m, ones(yc)))))
Y_b = kron(ones(3*mc*2*grd), y)

H_l = kron(nd,ones(2*yc*mc*grd))
I_l = kron(ones(3),kron(i,ones(yc*mc*grd)))
A_l = kron(ones(3), kron(2,kron(a_l, ones(yc*mc))))
M_l = kron(ones(3), kron(2, kron(ones(grd), kron(m, ones(yc)))))
Y_l = kron(ones(3*mc*2*grd), y)


9000-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [36]:
# a and i for solve
amat_b = [a_b a_b]
imat_b = [zeros(grd) ones(grd)]
ut_b   = [zeros(grd) ones(grd)]

100×2 Matrix{Float64}:
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 ⋮    
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0

In [37]:
amat_l = [a_l a_l]
imat_l = [zeros(grd) ones(grd)]
ut_l   = [zeros(grd) ones(grd)]

#Guess premium
prem = 0.1401
δ    = 0.7


0.7

In [39]:
dim1 = trunc(Int64, 3*2*yc*mc*grd)
dim2 = trunc(Int64, 1/3*dim1)
tolerance = 0.0001


0.0001

In [40]:
function solve()
dim1 = trunc(Int64, 3*2*yc*mc*grd)
dim2 = trunc(Int64, 0.5*dim1)
v_0b =  util.(A_b,A_b,Y_b,I_b,I_b, M_b,H_b)
v_0l =  util.(A_l,A_l,Y_l,I_l,I_l, M_l,H_l)
v_1b   = ones(dim1)
v_1l   = ones(dim1)
tolerance = 0.0001
v_b      = zeros(grd,2)
v_l      = zeros(grd,2)
policy0_b = zeros(dim1,2)
policy0_l = zeros(dim1,2)
policy_b = ones(dim1,2)
control_b = deepcopy(v_0)
cnt_pol_b = deepcopy(policy0)
policy_l = ones(dim1,2)
control_l = deepcopy(v_0)
cnt_pol_l = deepcopy(policy0)
iter = 0
while maximum(abs.(policyb.- policy0b)) & maximum(abs.(policyl.- policy0l))> tolerance
v_0b    =  deepcopy(controlb)
v_0l    =  deepcopy(controll)
policy0_b = deepcopy(cnt_polb)
policy0_l = deepcopy(cnt_poll)
for k in 1:dim1
con_b = consb.(A_b[k],amat_b,Y_b[k],I_b[k],imat_b, M_l[k],H_b[k])
con_l = consl.(A_l[k],amat_l,Y_l[k],I_l[k],imat_l, M_l[k],H_l[k])
checc_b = con_b.<=0
checc_l = con_l.<=0
matr_b    = checc_b*-1e6
matr_l    = checc_l*-1e6
ut_b =  (con_b.^(1-σ).-1)./(1-σ)
ut_l =  (con_l.^(1-σ).-1)./(1-σ)
ut_b = matr_b+ut_b
ut_l = matr_l+ut_l
for j in 1:2
for h in 1:grd
    v_b[h,j]=  ut_b[h,j]+ β*maximum(Prob_b[k,1+dim2*i[j]+mc*yc*(h-1):dim2*i[j]+mc*yc*(h)]'*v_0b[1+dim2*i[j]+yc*mc*(h-1):dim2*i[j]+yc*mc*(h)],Prob_l[k,1+dim2*i[j]+mc*yc*(h-1):dim2*i[j]+mc*yc*(h)]'*v_0l[1+dim2*i[j]+yc*mc*(h-1):dim2*i[j]+yc*mc*(h)])
    v_l[h,j]=  ut_l[h,j]+ β*maximum(Prob_b[k,1+dim2*i[j]+mc*yc*(h-1):dim2*i[j]+mc*yc*(h)]'*v_0b[1+dim2*i[j]+yc*mc*(h-1):dim2*i[j]+yc*mc*(h)],Prob_l[k,1+dim2*i[j]+mc*yc*(h-1):dim2*i[j]+mc*yc*(h)]'*v_0l[1+dim2*i[j]+yc*mc*(h-1):dim2*i[j]+yc*mc*(h)])
end
end
v_1b[k] = maximum(v_b)
v_1l[k] = maximum(v_l)
index_b =  findall(v_b.==maximum(v_b))[1]
index_l =  findall(v_l.==maximum(v_l))[1]
policy_b[k,1] = a_b[index[1]]
policy_l[k,1] = a_l[index[1]]
policy_b[k,2]  = i[index[2]]
policy_l[k,2]  = i[index[2]]

end
control_b = deepcopy(v_1b)
control_l = deepcopy(v_1l)
cnt_polb = deepcopy(policy_b)
cnt_poll = deepcopy(policy_l)
iter = iter+1
println(max(maximum(abs.(v_1b.-v_0b)),maximum(abs.(v_1b.-v_0b))))
end
return v_1b, v_1l, policy_b, policy_l, iter
end


solve (generic function with 1 method)

In [41]:
result   = solve()
v_1b     = result[1]
v_1l     = result[2]
policy_b = result[3]
policy_l = result[4]
iter_1   = result[5]

LoadError: ignored